<a href="https://colab.research.google.com/github/kgpark88/deeplearning/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ABC 데이터셋을 활용하여 이미지 분류기를 만들고 검증손실이 최소인 모델을 파일로 저장하세요.
## [조건]  
- 이미지 높이: 224픽셀
- 이미지 길이: 224픽셀
- 색상 채널: 3
- 모델파일명: abc.model

## 라이브러리 임포트(import)

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16


## 학습데이터와 검증데이터셋 준비
## [데이터셋 종류]
- 사람 표정 이미지
- 제품 양품 불량 이미지

In [ ]:
dataset_name = 'cats_vs_dogs'
train_dataset = tfds.load(name=dataset_name, split='train[:80%]')
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

# 이미지 변환 함수 작성

In [3]:
def preprocess(data):
    x = data['image']
    y = data['label']
    x = x / 255
    x = tf.image.resize(x, size=(224, 224))
    return x, y

## 모델 구성/빌드(build)/저장

In [1]:
batch_size = 32
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable = False

model = Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
    tf.keras.layers.Dense(2, activation='softmax')
])

checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.fit(train_data,
          validation_data=(valid_data),
          epochs=20,
          callbacks=[checkpoint],
          )

NameError: ignored